'hihello'라는 문자열을 예측하는 모델을 생성</br>
예를들어 'h'가 들어오면 'i' or 'e'를 예측해야 한다.</br>
'i' or 'e'를 판단하기 위해서 문장의 어디쯤 들어왔는지를 모델에 알려줘야 한다.</br>

In [6]:
import torch
import torch.optim as optim
import numpy as np

# 결과를 재현 가능하게하는 random seed 설정 
torch.manual_seed(0)

# 입력 데이터를 만든다.
x_data = [[0, 1, 0, 2, 3, 3]]

# 문자를 단순한 숫자로 매핑하면 숫자의 크기에 별다른 의미가 없음에도 순서가 생겨버린다.
# 이럴때는 one-hot 인코딩을 써서 구분하고 순서가 존재하지 않도록 한다.
x_one_hot = [[[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 1, 0]]]
# ihello를 인덱스로 표현
y_data = [[1, 0, 2, 3, 3, 4]]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

# 존재하는 모든 문자의 set을 만든다.
char_set = ['h', 'i', 'e', 'l', 'o']

# input_size는 char의 개수만큼의 크기가 설정되어야 한다.
input_size = len(char_set)
# 여기서는 어떤 값을 주어도 관계없으나 문자의 개수로 설정했다.
hidden_size = len(char_set)
learning_rate = 0.1

# batch_first는 출력의 순서를 batch의 차원을 가장 앞에둔다. 출력순서 = (B, S, F)
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)  

# CrossEntropyLoss는 categorical한 output을 만드는 모델에서 많이 쓰인다.
criterion = torch.nn.CrossEntropyLoss()
# Adam optimizer를 사용한다.
optimizer = optim.Adam(rnn.parameters(), learning_rate)

# start training
for i in range(100):
    # 매 루프마다 새로운 gradient로 시작할 수 있다.
    optimizer.zero_grad()       
    # 여기서 _status는 hidden state이다. 여기서는 모든 입력을 넣어서 나온 결과라서 쓰이지 않는다.
    outputs, _status = rnn(X)   
    # view(-1)은 batch dimension이 앞에 오도록 바꾸어 준다.
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    # back propagation이 진행된다.
    loss.backward()
    # 최적화를 진행해서 파라미터들을 업데이트한다.
    optimizer.step()

    # argmax 함수는 값이 가장 큰 인덱스만 가져온다. 
    # 3번째 차원의 값들중 가장 큰 값의 인덱스를 가져온다. 
    result = outputs.data.numpy().argmax(axis=2)
    # result의 인덱스가 어떤 문자에 해당하는지 가져와서 하나의 문자열로 합친다.
    # squeeze 함수는 shape에서 demension이 1인 축을 없애준다.
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.7802648544311523 prediction:  [[1 1 1 1 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  iiiiii
1 loss:  1.4931949377059937 prediction:  [[1 4 1 1 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ioiioo
2 loss:  1.3337111473083496 prediction:  [[1 3 2 3 1 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ilelio
3 loss:  1.2152947187423706 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elelll
4 loss:  1.1131387948989868 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elelll
5 loss:  1.0241864919662476 prediction:  [[2 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elello
6 loss:  0.9573140740394592 prediction:  [[2 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elello
7 loss:  0.9102001786231995 prediction:  [[2 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ehello
8 loss:  0.8731765151023865 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
9